In [36]:
import cv2
from glob import glob
import pathlib 
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [37]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten,Dense,Conv3D,MaxPool3D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
df=pathlib.Path("/content/drive/MyDrive/CNN/Dataset")

In [40]:
zero_list=list(df.glob("Dantu/*.jpg"))
one_list=list(df.glob('Jackfruit/*.jpg'))
two_list=list(df.glob('Neem/*.jpg'))
three_list=list(df.glob('Basale/*.jpg'))
four_list=list(df.glob('Mustard/*.jpg'))
five_list=list(df.glob('Lemon/*.jpg'))
six_list=list(df.glob('Roxburgh/*.jpg'))
seven_list=list(df.glob('Peepal/*.jpg'))
eight_list=list(df.glob('Rosa/*.jpg'))
nine_list=list(df.glob('Jasmine/*.jpg'))
     # *-shows all #(malignant/*)- shows all files inside the malignant folder


In [41]:
len(zero_list),len(one_list),len(two_list),len(three_list),len(four_list),len(five_list),len(six_list),len(seven_list),len(eight_list),len(nine_list)

(122, 56, 67, 103, 34, 57, 50, 63, 43, 71)

In [42]:
disease_dict={"Dantu":zero_list,
         "Jackfruit":one_list,
         "Neem":two_list,
         "Basale":three_list,
         "Mustard":four_list,
         "Lemon":five_list,
         "Roxburgh":six_list,
         "Peepal":seven_list,
         "Rosa":eight_list,
         "Jasmine":nine_list}
disease_class={"Dantu":0,
         "Jackfruit":1,
         "Neem":2,
         "Basale":3,
         "Mustard":4,
         "Lemon":5,
         "Roxburgh":6,
         "Peepal":7,
         "Rosa":8,
         "Jasmine":9}         

In [43]:
x=[]
y=[]

In [44]:
for i in disease_dict:
  disease_name=i
  disease_path_list=disease_dict[disease_name]
  for path in disease_path_list:
    img=cv2.imread(str(path))
    img=cv2.resize(img,(224,224))
    img=img/255
    x.append(img)
    cls=disease_class[disease_name]
    y.append(cls)

In [45]:
len(x)

666

In [46]:
x=np.array(x)
y=np.array(y)

In [47]:
from sklearn.model_selection import train_test_split

In [48]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,train_size=0.75,random_state=1)

In [49]:
len(xtrain),len(ytrain),len(xtest),len(ytest)

(499, 499, 167, 167)

In [50]:
xtrain.shape

(499, 224, 224, 3)

In [51]:
xtrain.shape,xtest.shape

((499, 224, 224, 3), (167, 224, 224, 3))

In [52]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

In [53]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')

In [54]:
print("[INFO] summary for base model...")
print(base_model.summary())

[INFO] summary for base model...
Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )             

In [55]:
from tensorflow.keras.layers import MaxPooling2D
from keras.layers.core import Dropout
from tensorflow.keras.models import Model
# construct the head of the model that will be placed on top of the
# the base model
headModel = base_model.output
headModel = MaxPooling2D(pool_size=(2, 2))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(32, activation="relu")(headModel)
headModel = Dropout(0.2)(headModel)
headModel = Dense(10, activation="softmax")(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=base_model.input, outputs=headModel)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in base_model.layers:
	layer.trainable = False

In [60]:
from tensorflow.keras.optimizers import Adam
# compile our model (this needs to be done after our setting our
# layers to being non-trainable)
print("[INFO] compiling model...")
opt = Adam(lr=1e-4)
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
# initialized with actual "learned" values versus pure random
print("[INFO] training head...")
#H = model.fit(
	#data_generator.flow(xtrain, ytrain, batch_size=32),
#	steps_per_epoch=len(xtrain) // 32,
	#validation_data=valAug.flow(xtest, ytest),
	#validation_steps=len(xtest) // 32,
#	epochs=5)

model_hist=model.fit(xtrain,ytrain,epochs=15,validation_data=(xtest,ytest),batch_size=180)

[INFO] compiling model...
[INFO] training head...
Epoch 1/15
3/3 [==============================] - 84s 13s/step - loss: 0.5407 - accuracy: 0.8236 - val_loss: 0.1851 - val_accuracy: 0.9401
Epoch 2/15
3/3 [==============================] - 29s 11s/step - loss: 0.4924 - accuracy: 0.8076 - val_loss: 0.2476 - val_accuracy: 0.9222
Epoch 3/15
3/3 [==============================] - 29s 11s/step - loss: 0.4393 - accuracy: 0.8357 - val_loss: 0.2276 - val_accuracy: 0.9281
Epoch 4/15
3/3 [==============================] - 31s 12s/step - loss: 0.3616 - accuracy: 0.8677 - val_loss: 0.1146 - val_accuracy: 0.9760
Epoch 5/15
3/3 [==============================] - 31s 12s/step - loss: 0.3119 - accuracy: 0.8838 - val_loss: 0.1212 - val_accuracy: 0.9641
Epoch 6/15
3/3 [==============================] - 29s 10s/step - loss: 0.3000 - accuracy: 0.8858 - val_loss: 0.1069 - val_accuracy: 0.9701
Epoch 7/15
3/3 [==============================] - 38s 15s/step - loss: 0.2873 - accuracy: 0.8878 - val_loss: 0.1065 

In [61]:
model.save("heart.h5")

In [62]:
from tensorflow.keras.preprocessing import image
# testing the model
def testing_image(image_directory):
    test_image = image.load_img(image_directory, target_size = (224, 224))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    test_image = test_image/255
    result = model.predict(x= test_image)
    print(result)
    if np.argmax(result)==0:

       prediction='Dantu'
    elif np.argmax(result)==1:        
       prediction='Jackfruit' 
    elif np.argmax(result)==2:
      prediction='Neem'
    elif np.argmax(result)==3:
      prediction='Basale'
    elif np.argmax(result)==4:
      prediction='Mustard'
    elif np.argmax(result)==5:
      prediction='Lemon'
    elif np.argmax(result)==6:
      prediction='Roxburgh'
    elif np.argmax(result)==7:
      prediction='Peepal'
    elif np.argmax(result)==8:
      prediction='Rosa'
    else:
      prediction='Jasmine'  
    return prediction                 

In [63]:
print(testing_image('/content/drive/MyDrive/CNN/Dataset/Dantu/AV-S-002.jpg'))

1/1 [==============================] - 1s 1s/step
[[9.9994099e-01 3.1440450e-05 6.0477096e-10 1.1920305e-06 2.2139648e-07
  1.4156859e-08 1.3637082e-11 2.1751242e-05 2.1544655e-10 4.4663966e-06]]
Dantu
